In [8]:
import numpy as np #linear algebra
import pandas as pd #data manipulation and analysis
import seaborn as sns #data visualization
import matplotlib.pyplot as plt #data visualization
import sklearn.preprocessing as skp #machine learning (preprocessing)
import sklearn.cluster as skc #machine learning (clustering)
import warnings # ignore warnings
warnings.filterwarnings('ignore')

path = "../data/cleanData/fullData.csv"

In [9]:
df = pd.read_csv(path)
df = df.rename(columns = {'Unnamed: 0':'County Name'})
df=df.fillna(0)
df.head()

,County Name,"Population Estimates, July 1 2021, (V2021)",Persons under 5 years,Persons under 18 years,Persons 65 years and over,Female persons,White alone,Black or African American alone,American Indian and Alaska Native alone,Asian alone,...,"Mean travel time to work (minutes), workers age 16 years+, 2017-2021","Median household income (in 2021 dollars), 2017-2021","Per capita income in past 12 months (in 2021 dollars), 2017-2021",Persons in poverty,"Total employer establishments, 2020","Total employment, 2020","Total annual payroll, 2020 ($1,000)","Population per square mile, 2020","Land area in square miles, 2020",Banned or not
0,"Adams County, Ohio",27542,1707.60,6665.16,5095.27,13798.54,26660.66,165.25,165.25,82.63,...,36.1,44467,24539,5205.44,385,4572,167427,47.1,583.87,0.0
1,"Allen County, Ohio",101670,6100.20,23587.44,18503.94,50021.64,84081.09,12912.09,305.01,915.03,...,19.5,55114,28671,15555.51,2286,45012,1983272,253.9,402.55,1.0
2,"Ashland County, Ohio",52316,2929.70,11666.47,10201.62,26524.21,50484.94,470.84,156.95,418.53,...,24.2,58168,28992,5702.44,1034,18234,701075,124.0,422.99,0.0
3,"Ashtabula County, Ohio",97337,5450.87,21414.14,19467.40,47889.80,90231.40,3796.14,389.35,486.68,...,25.8,49680,26777,15281.91,1806,24464,913850,139.0,702.07,0.0
4,"Athens County, Ohio",62056,2296.07,8998.12,8874.01,31090.06,56657.13,1799.62,248.22,1737.57,...,21.9,47061,24990,12969.70,1029,13265,469095,124.0,503.64,0.0


In [2]:
# Running kmean with only these three vars.

In [3]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.feature_selection import SelectKBest, mutual_info_classif

# Generate a fake dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=5, n_redundant=0, random_state=42)

# Convert X and y to pandas dataframes
X = df_cluster_id12.iloc[:, :-3]
y = df_cluster_id12['Banned or not']

# Use mutual information to select the top 5 most important variables
selector = SelectKBest(mutual_info_classif, k=3)
selector.fit(X, y)

# Get the top 5 most important variables
top_vars = X.columns[selector.get_support()]
print('The top 3 most important variables are:')
print(top_vars)

NameError: name 'df_cluster_id12' is not defined

In [ ]:
# Create new data frame for Clustering
df_clusterTest = df[['Persons 65 years and over', 'Housing units, July 1, 2021, (V2021)',
       'Per capita income in past 12 months (in 2021 dollars), 2017-2021']] # No binary Var
df_clusterTest

In [ ]:
# Scaling the new data frame for clustering
sc = skp.StandardScaler()
data_scale = np.array(df_clusterTest)
scaled = sc.fit_transform(data_scale.astype(float))
scaled_dfTest = pd.DataFrame(scaled, columns=df_clusterTest.columns)
scaled_dfTest.head()

In [ ]:
###Decide n-cluster using Elbow Method
wcss=[]
k_range = range(1,11)
for i in k_range:
    kmeans = skc.KMeans(n_clusters=i, init='k-means++', random_state=42)
    kmeans.fit(scaled_dfTest)
    wcss.append(kmeans.inertia_)
fig, ax = plt.subplots(figsize=(8, 6), dpi=80)
plt.plot(k_range, wcss, marker='o')
# for i, value in enumerate(wcss):
#     ax.text(i+1.05, value-0.005, round(value,1), fontsize=12, fontweight='bold')
    
plt.xticks(k_range)
# plt.grid()
plt.title('The Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.show()

In [ ]:
# Clustering K Means, K=3
kmeans_3 = skc.KMeans(n_clusters=3,random_state=42)
kmeans_3.fit(scaled_dfTest)
kmeans_3.labels_

In [ ]:
# Assign clustering result to each country in the data frame
df_clusterTest['cluster_id'] = kmeans_3.labels_
df_clusterTest

In [ ]:
df_clusterTest['Banned or not'] = df.iloc[:,-1]
df_clusterTest['County Name'] = df.iloc[:,0]
df_clusterTest

In [ ]:
groupeddfTest = df_clusterTest.groupby('cluster_id')['Banned or not'].value_counts(ascending=True)
groupeddfTest

In [ ]:
# Change default style
sns.set_style('whitegrid')

# Centroid Inverse Scaling
centroids_ori_scale = sc.inverse_transform(kmeans_3.cluster_centers_)
# Plot Clustering Result
fig, ax = plt.subplots(figsize=(12, 7), dpi=200)
sns.scatterplot(df_clusterTest.loc[df_clusterTest['cluster_id'] == 0, df_clusterTest.columns[0]],
                df_clusterTest.loc[df_clusterTest['cluster_id'] == 0, df_clusterTest.columns[1]],
                color = 'green', s=100, edgecolor = 'green', label= '0',
                style=df_cluster['Banned or not'])

sns.scatterplot(df_clusterTest.loc[df_clusterTest['cluster_id'] == 1, df_clusterTest.columns[0]],
                df_clusterTest.loc[df_clusterTest['cluster_id'] == 1, df_clusterTest.columns[1]],
                color = 'red', s=100, edgecolor = 'red', label= '1',
                style=df_cluster['Banned or not'])

sns.scatterplot(df_clusterTest.loc[df_clusterTest['cluster_id'] == 2, df_clusterTest.columns[0]],
                df_clusterTest.loc[df_clusterTest['cluster_id'] == 2, df_clusterTest.columns[1]],
                color = 'blue', s=100, edgecolor = 'blue', label= '2',
                style=df_cluster['Banned or not'])

#ax.scatter(centroids_ori_scale[:, -7], centroids_ori_scale[:,0], c='black', s=400, marker='o', alpha=0.6, label = 'centroid')
plt.legend(title= "Cluster ID", labelspacing=1.5, borderpad=1)
plt.xlabel(df_cluster.columns[-7])
plt.ylabel(df_cluster.columns[0])
plt.title(f"Clustering {df_cluster.columns[-7]} & {df_cluster.columns[0]}")
plt.savefig('cluster_result.png')
plt.show()

In [4]:
# Change default style
sns.set_style('whitegrid')
for i in range(len(df_clusterTest.iloc[:, :-3].columns)):
    for j in range(len(df_clusterTest.iloc[:, :-3].columns)):
        if df_clusterTest.iloc[:, :-3].columns[i] != df_clusterTest.iloc[:, :-3].columns[j]:
            # Centroid Inverse Scaling
            centroids_ori_scale = sc.inverse_transform(kmeans_3.cluster_centers_)
            # Plot Clustering Result
            fig, ax = plt.subplots(figsize=(12, 7), dpi=200)
            sns.scatterplot(df_clusterTest.loc[df_clusterTest['cluster_id'] == 0, df_clusterTest.iloc[:, :-3].columns[i]],
                            df_clusterTest.loc[df_clusterTest['cluster_id'] == 0, df_clusterTest.iloc[:, :-3].columns[j]],
                            color = 'green', s=100, edgecolor = 'green', label= '0',
                            style=df_cluster['Banned or not'])

            sns.scatterplot(df_clusterTest.loc[df_clusterTest['cluster_id'] == 1, df_clusterTest.iloc[:, :-3].columns[i]],
                            df_clusterTest.loc[df_clusterTest['cluster_id'] == 1, df_clusterTest.iloc[:, :-3].columns[j]],
                            color = 'red', s=100, edgecolor = 'red', label= '1',
                            style=df_cluster['Banned or not'])

            sns.scatterplot(df_clusterTest.loc[df_clusterTest['cluster_id'] == 2, df_clusterTest.iloc[:, :-3].columns[i]],
                            df_clusterTest.loc[df_clusterTest['cluster_id'] == 2, df_clusterTest.iloc[:, :-3].columns[j]],
                            color = 'blue', s=100, edgecolor = 'blue', label= '2',
                            style=df_cluster['Banned or not'])

            #ax.scatter(centroids_ori_scale[:, -7], centroids_ori_scale[:,0], c='black', s=400, marker='o', alpha=0.6, label = 'centroid')
            plt.legend(title= "Cluster ID", labelspacing=1.5, borderpad=1)
            plt.xlabel(df_clusterTest.columns[i])
            plt.ylabel(df_clusterTest.columns[j])
            plt.title(f"Clustering {df_clusterTest.columns[i]} & {df_clusterTest.columns[j]}")
            plt.savefig('cluster_result.png')
            plt.show()

NameError: name 'sns' is not defined

In [5]:
# top pairs

In [6]:
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LinearRegression

# Split data into features and target
# Convert X and y to pandas dataframes
X = df_cluster_id12.iloc[:, :-3]
y = df_cluster_id12['Banned or not']

# Use RFECV to select the top 3 pairs of features
estimator = LinearRegression()
selector = RFECV(estimator, step=1, cv=5)
selector = selector.fit(X, y)

# Get the indices of the top 3 pairs of features
top_pairs = np.argsort(selector.ranking_)[:3]
#top_pairs = [tuple(X.columns[pair]) for pair in top_pairs]
print(top_pairs)


NameError: name 'df_cluster_id12' is not defined

In [7]:
for p in top_pairs:
    print(X.columns[p])

NameError: name 'top_pairs' is not defined